In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Lupus_(Systemic_Lupus_Erythematosus)"
cohort = "GSE193442"

# Input paths
in_trait_dir = "../../input/GEO/Lupus_(Systemic_Lupus_Erythematosus)"
in_cohort_dir = "../../input/GEO/Lupus_(Systemic_Lupus_Erythematosus)/GSE193442"

# Output paths
out_data_file = "../../output/preprocess/Lupus_(Systemic_Lupus_Erythematosus)/GSE193442.csv"
out_gene_data_file = "../../output/preprocess/Lupus_(Systemic_Lupus_Erythematosus)/gene_data/GSE193442.csv"
out_clinical_data_file = "../../output/preprocess/Lupus_(Systemic_Lupus_Erythematosus)/clinical_data/GSE193442.csv"
json_path = "../../output/preprocess/Lupus_(Systemic_Lupus_Erythematosus)/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Transcriptional profiling of human KIR+ CD8 T cells"
!Series_summary	"This SuperSeries is composed of the SubSeries listed below."
!Series_overall_design	"Refer to individual Series"
Sample Characteristics Dictionary:
{0: ['tissue: PBMC'], 1: ['cell type: KIR+ CD8 T']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# Analyze the dataset based on the background information and sample characteristics

# 1. Gene Expression Data Availability
# Based on the series title and sample characteristics, this dataset seems to focus on transcriptional profiling
# of human KIR+ CD8 T cells, which suggests it contains gene expression data.
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# From the sample characteristics, we don't see explicit fields for lupus/SLE status, age, or gender
# The dataset appears to be focusing only on cell types rather than patient characteristics

# For trait (SLE)
# No explicit SLE status is provided in the sample characteristics
trait_row = None  # No explicit trait information available

# For age
# No age information is provided in the sample characteristics
age_row = None  # No age information available

# For gender
# No gender information is provided in the sample characteristics
gender_row = None  # No gender information available

# 2.2 Data Type Conversion
# Since we don't have access to these variables, we'll define placeholder conversion functions

def convert_trait(value):
    # Placeholder function since trait data is not available
    if value is None:
        return None
    
    value = value.split(': ')[-1].strip().lower()
    if 'lupus' in value or 'sle' in value:
        return 1
    elif 'control' in value or 'healthy' in value or 'normal' in value:
        return 0
    else:
        return None

def convert_age(value):
    # Placeholder function since age data is not available
    if value is None:
        return None
    
    try:
        # Extract value after colon and convert to float
        age_str = value.split(': ')[-1].strip()
        return float(age_str)
    except (ValueError, AttributeError):
        return None

def convert_gender(value):
    # Placeholder function since gender data is not available
    if value is None:
        return None
    
    value = value.split(': ')[-1].strip().lower()
    if 'female' in value or 'f' == value:
        return 0
    elif 'male' in value or 'm' == value:
        return 1
    else:
        return None

# 3. Save Metadata
# Check if trait data is available
is_trait_available = trait_row is not None

# Save the initial filtering result
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Since trait_row is None, we skip the clinical feature extraction step


False

### Step 3: Gene Data Extraction

In [4]:
# 1. First get the path to the soft and matrix files
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Looking more carefully at the background information
# This is a SuperSeries which doesn't contain direct gene expression data
# Need to investigate the soft file to find the subseries
print("This appears to be a SuperSeries. Looking at the SOFT file to find potential subseries:")

# Open the SOFT file to try to identify subseries
with gzip.open(soft_file, 'rt') as f:
    subseries_lines = []
    for i, line in enumerate(f):
        if 'Series_relation' in line and 'SuperSeries of' in line:
            subseries_lines.append(line.strip())
        if i > 1000:  # Limit search to first 1000 lines
            break

# Display the subseries found
if subseries_lines:
    print("Found potential subseries references:")
    for line in subseries_lines:
        print(line)
else:
    print("No subseries references found in the first 1000 lines of the SOFT file.")

# Despite trying to extract gene data, we expect it might fail because this is a SuperSeries
try:
    gene_data = get_genetic_data(matrix_file)
    print("\nGene data extraction result:")
    print("Number of rows:", len(gene_data))
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")
    print("This confirms the dataset is a SuperSeries without direct gene expression data.")

This appears to be a SuperSeries. Looking at the SOFT file to find potential subseries:
No subseries references found in the first 1000 lines of the SOFT file.



Gene data extraction result:
Number of rows: 0
First 20 gene/probe identifiers:
Index([], dtype='object', name='ID')
